In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd

import itertools

from py_wake.deficit_models import EddyViscosityDeficitModel, EddyViscosityModel, BastankhahGaussianDeficit
from py_wake import HorizontalGrid

# turbines and sites
from py_wake.site import XRSite, UniformWeibullSite, UniformSite
from py_wake.site._site import Site
from py_wake.site.shear import PowerShear
from py_wake.wind_turbines import WindTurbine
from py_wake.wind_turbines.generic_wind_turbines import GenericWindTurbine
from py_wake.examples.data.hornsrev1 import Hornsrev1Site, V80
from py_wake.examples.data.iea37 import IEA37Site, IEA37_WindTurbines
from py_wake.examples.data.ParqueFicticio import ParqueFicticioSite
from py_wake.examples.data.dtu10mw import DTU10MW

from py_wake.wind_farm_models.wind_farm_model import SimulationResult
from py_wake.wind_farm_models import PropagateDownwind
from py_wake.wind_turbines import WindTurbine, WindTurbines
from py_wake.wind_turbines.power_ct_functions import PowerCtTabular

#TODO move this function in utils
def my_arange(start, end, step):
    '''Function for np.arange(start, stop, step) without creating problems with the float representations'''
    factor = 10
    while factor*step < 1:
        factor *= 10
    return np.arange(start*factor, end*factor, step*factor) / factor

# Preparation of the parameters for the data generation

## Site (to set turbulence intensity TI and...?)
For a given position, reference wind speed (WSref) and wind direction (WDref), Site provides the local wind condition in terms of wind speed (WS), wind direction (WD), turbulence intensity (TI) and the probability of each combination of wind direction and wind speed. Furthermore, Site is responsible for calculating the down-wind, cross-wind and vertical distance between wind turbines (which in non-flat terrain is different from the straight-line distances).

In [ ]:
"""
def OLD_get_sites(turbulence_intensity : float) -> list[XRSite]:

    # assert 0 <= turbulence_intensity <= 1 #TODO if it is a percentage, this is correct

    custom_site = UniformWeibullSite(
        p_wd = [.20,.25,.35,.25],                         # sector frequencies
        a = [9.176929,  9.782334,  9.531809,  9.909545],  # Weibull scale parameter
        k = [2.392578, 2.447266, 2.412109, 2.591797],     # Weibull shape parameter
        ti = turbulence_intensity,                        # turbulence intensity, optional
    )
    '''
    not working
    custom_site = XRSite(
        ds=xr.Dataset(data_vars={'WS': range(3, 26), 'P': ('wd', [1/360] * len(range(0, 360))), 'TI': 0.25},
                    coords={'wd': range(0, 360)}),
        shear=PowerShear(h_ref=100, alpha=.2)
    )
    '''
    custom_site = XRSite(ds=xr.Dataset(data_vars={'WS': 5, 'P': 1, 'TI': 0.1}))
                    #coords={'wd': range(0, 360)}
    return [IEA37Site(ti=turbulence_intensity)] #, Hornsrev1Site(ti=turbulence_intensity), custom_site] #ParqueFicticioSite() not having ti as an argument
"""
    
def get_site(ti: float, ws: int) -> XRSite:
    #return XRSite(ds=xr.Dataset(data_vars={'WS': WS_RANGE, 'P': 1, 'TI': turbulence_intensity}, coords={'wd': WD_RANGE}))
    return UniformSite(ti=ti, ws=ws)

## Turbine (to set thrust coefficient CT)

For a given wind turbine type and effective wind speed (WSeff), the WindTurbine object provides the power and thrust coefficient (CT), as well as the wind turbine hub height (H) and diameter (D).


In [ ]:
"""
def OLD_get_wind_turbines() -> list[WindTurbine]:
    '''option 1 (obsolete)'''
    # you need 3 parameters for a custom turbine: u (ws_values), power and ct curves
    # (see how powerCtFunction is initialized in the pre-defined turbines)
    u: list[int] = [5, 10, 15, 20, 25]
    power: list[float] = [1000, 1000, 1000, 1000, 1000]
    ct: list[float] = [0.99, 0.5, 0.75, .3, 0.9]
    custom_turbine = WindTurbine(name="custom", diameter=100, hub_height=100, powerCtFunction=PowerCtTabular(u, power,'kW', ct))

    '''implemented options (to delete)'''
    return [V80(), IEA37_WindTurbines(), DTU10MW()]  #, custom_turbine] 
"""

def get_wind_turbine(ct: float, ti: float) -> WindTurbine:
    # diameter, hub height and power norm taken from IEA37 as these params affect the simulation
    # TODO update values or try different ones?
    diameter = 198
    hub_height = 119
    power_norm = 10000
    return GenericWindTurbine(name="AinslieTurbine",
                              diameter=diameter, hub_height=hub_height, power_norm=power_norm,
                              constant_ct=ct, turbulence_intensity=ti,
                              # ws_lst=np.arange(4, 26, 1), ws_cutin=4, ws_cutout=25 TODO set these parameters for the power ct function
                              )



## Discretization

In [ ]:
def get_discretized_grid(diameter: int, grid_step_factor: float) -> HorizontalGrid:
    # TODO check the discretization (see Javier thesis, appendix B in particular), my current reasonings:
    # FOR X RANGE
    # the start is diameter*2 to skip the near-wake
    #       (careful with the wind direction, if it is within the whole range (0, 360), near-wake intercepted)
    # the end is diameter*50 (i.e. DEFAULT_MAXIMUM_WAKE_DISTANCE in PyWake EddyViscosity definition) #TODO reduce it
    # the step is parametrized (Javier's thesis trying different values, e.g. diameter/2, diameter/4, ..., diameter/16)

    # FOR Y RANGE
    # since the wind direction remains stable at 270, it is useless to put a big range for y
    
    #TODO see also datadriven wind turbine wake modelling via probabilistic ML (e.g. Fig. 3) to set these parameters
    
    x_range = np.arange(diameter*2, diameter*50, diameter*grid_step_factor)
    y_range = np.arange(-diameter, diameter, diameter*grid_step_factor)
    return HorizontalGrid(x = x_range, y = y_range)

# Actual data generation

In [ ]:
def generate_wake_dataframe(model, wind_speed: float, wind_direction: float,
                            wind_turbine: WindTurbine, turbine_xs: list[int], turbine_ys: list[int],
                            grid_step_factor: float
                            ) -> pd.DataFrame:

    diameter = wind_turbine.diameter().item() # this is used for scaling distances and to exclude near-wake region
    grid = get_discretized_grid(diameter, grid_step_factor)
    
    # the flow_map creates a warning for near-wake calculations
    sim_res = model(
        x=turbine_xs, y=turbine_ys, # wind turbine positions (setting also wt domain, i.e. the number of turbines)
        wd=wind_direction,          # Wind direction (None for default -> 0-360° in bins of 1°)
        ws=wind_speed,              # Wind speed (None for default -> 3-25 m/s in bins of 1 m/s)
        #yaw=0                      # TODO try to change this parameter?
        #h=None,                    # wind turbine heights (defaults to the heights defined in windTurbines)
        #type=0,                    # Wind turbine types
    )
    flow_map = sim_res.flow_map(grid)
    
    # removing h, wd and ws
    h = flow_map['h'].item()
    ti = sim_res.TI.item()
    #print(f"ti in generation: {ti}")
    flow_map = flow_map\
        .sel(h=h)\
        .sel(wd=wind_direction)\
        .sel(ws=wind_speed)
    flow_map = flow_map.drop_vars(["h", "wd", "ws", "WD", "TI", "P"])

    flow_map["ti"] = xr.DataArray([ti], dims="ti")
    # ct is now correlated to the wind speed (which would make it useless for the prediction) but it should depend also on the turbine
    ct = wind_turbine.ct(ws=wind_speed) #TODO
    #print(f"ct in generation: {ct}")
    flow_map["ct"] = xr.DataArray([ct], dims="ct") #TODO putting this as a dimension is wrong and creates problems in the xarray (this is why dataframe)

    #TODO sistema codice di seguito
    df = flow_map.to_dataframe()
    df["wind_deficit"] = 1 - df["WS_eff"]/df["WS"]
    # scaling x and y according to the diameter
    df.reset_index(inplace=True)
    df["x/D"] = df["x"] / diameter
    df["y/D"] = df["y"] / diameter
    df.drop(["x", "y"], axis=1, inplace=True)
    df.set_index(["x/D", "y/D", "ti", "ct"], inplace=True) #TODO ???

    return df

In [ ]:
# default parameters
TURBINE_X = [0]
TURBINE_Y = [0]
GRID_STEP_DIAM_FACTOR = 1/16
WIND_DIRECTION = 270 # the wind turbine's yaw angle is always adjusted according to the wind direction, thus it is probably useless to generate data with more wind directions
#WD_RANGE = range(0, 360) # (by default, 0-360° in bins of 1°)

# parameters for data generation TODO check which right values to use for generation
#WS_RANGE = range(3, 26) # by default, 3-25 m/s in bins of 1 m/s
WS_RANGE = range(3, 26) # this can be set to one value (if the velocity does not affect the resulting wake fields) or a range
TIs = my_arange(0, 1, 0.01) # the ti is percentage
CTs = my_arange(0, 1, 0.01)
# according to the "AeroDyn Theory Manual", the maximum CT should be 24/25 -> see beginning of https://onlinelibrary.wiley.com/doi/full/10.1002/we.2688

# filepath
DATAFRAME_FILEPATH = f"data/wake_dataframe_{GRID_STEP_DIAM_FACTOR}diam.csv"

In [ ]:
dataframes = list()

for wind_speed, ti, ct in itertools.product(WS_RANGE, TIs, CTs):
    print(f"\r{wind_speed=}\t{ti=}\t{ct=}", end="\r")
    site = get_site(ti=ti, ws=wind_speed)
    wind_turbine = get_wind_turbine(ct=ct, ti=ti) #TODO THIS DOES NOT SET THE CONSTANT CT YET

    # single wake model
    ainslie_model = EddyViscosityModel(site, wind_turbine)

    df = generate_wake_dataframe(ainslie_model, wind_speed, WIND_DIRECTION,
                                wind_turbine, TURBINE_X, TURBINE_Y,
                                GRID_STEP_DIAM_FACTOR)
    #TODO add control to make sure that I do not add the same input variables twice
    dataframes.append(df)

final_df = pd.concat(dataframes)
final_df

In [ ]:
#final_df.to_csv(DATAFRAME_FILEPATH)